### Import

In [21]:
import os
import csv
import json
import shutil
from io import StringIO
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path
import matplotlib.pyplot as plt

import cv2
from PIL import Image
import pytesseract

import fitz  # Módulo PyMuPDF
import re
from fuzzywuzzy import fuzz
from unidecode import unidecode
from unicodedata import normalize
import PyPDF2

import pandas as pd

from PIL import ImageFont
from PIL import Image, ImageDraw

from datetime import datetime, timezone, timedelta

from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

import modules.extrai_pdf_pesquisavel as Extc
import modules.cronometro as cron

### Config

In [23]:
### PRESTAR ATENCAO
model = 'pedro_aldeia'


#### IMPORTANTE - Nro Batch
batch_name = "Batch_15"


# 1. path para documentos PDF (omelhor se estiverem dentro de um unico diretorio)
root_pdf_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 2. path para documentos PDF que podem estar aguardando para serem processados
root_pdf_aguardando_path = "pipeline_extracao_documentos/3_tratamento_excecoes/pdf_aguardando_processar"

documentos_extracao_path = "pipeline_extracao_documentos/2_documentos_para_extracao"

# 3. path para documentos PDF externos para serem processados
root_external_pdf_path = "content_from_pdftool/data/data_pdf/NF_para_processamento/NFRJ_PDF_para _ocr"
# 4. path para documentos PDF PESQUISAVEIS externos para serem processados
root_external_pdf_pesquisavel_path = "content_from_pdftool/data/data_pdf/NF_processadas/NFRJ/fwdnotasfiscaisemitidaslmpadalegal"

# 5. path para imagem padrao
image_resized_path = 'pipeline_extracao_documentos/6_geral_administacao/images/processadas'

# 6. path para log
log_path = 'pipeline_extracao_documentos/6_geral_administacao/logs'

# 7. path para arquivos json
json_path = "pipeline_extracao_documentos/5_documentos_processados/jsons"

# 8. path para NFs processadas
nf_processada_path = "pipeline_extracao_documentos/5_documentos_processados"

#### paths de objetos para criacao/gestao (dicionarios/datasets)
# 9. path para models
nf_model_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/frames_nf_v9.xlsx"

# 10. path para dicionario de models
model_dict_path = "pipeline_extracao_documentos/6_geral_administacao/modelos/models.csv"

# 11. path para datasets CNAE e Itens de Serviço
nf_datasets_path = "pipeline_extracao_documentos/6_geral_administacao/datasets"



# VERIFICAR
tgt_imagens = "pipeline_extracao_documentos/6_geral_administacao/images"

# 12. poppler path
poppler_path = "/home/dani-boy/miniconda3/envs/tables-detr/bin"

# 13. path para config Tesseract
tessdata_dir_config = '--tessdata-dir "/home/dani-boy/miniconda3/envs/tables-detr/share/tessdata/" --user-patterns "novo_modelo/modelos/user-patterns2.txt" --dpi 600 --oem 3 --psm 6'


### Tratamento template do modelo

In [24]:
#Le a planilha e cria do DF
frames_nf_v4_df = pd.read_excel(nf_model_path)

# Cria dicionários para armazenar diferentes tipos de elementos do modelo
document_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'document'].iloc[0]
boundaries_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'boundaries']
sections_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'section']
frames_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'frame']
sframe_fields_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'sframe_field']
field_boxes_info = frames_nf_v4_df[frames_nf_v4_df['type'] == 'field_box']

In [25]:
frames_nf_v4_df

,id,model,seq,prestador,type,color,f_size,box,t_value,father,label,section_json,reference,x0,y0,x1,y1,Largura,Altura,%
0,1,mage,1,NaN,document,NaN,NaN,NaN,NaN,NaN,modelo_prefeitura_mage,NaN,NaN,0.0,0.0,2067.0,2923.0,2067.0,2923.0,NaN
1,2,mage,2,todos,boundaries,green,120.0,yes,NaN,modelo_prefeitura_mage,boundaries_modelo_prefeitura_mage,NaN,NaN,144.0,99.0,1925.0,2666.0,1781.0,2567.0,100.000000
2,3,mage,3,todos,section,orange,110.0,yes,NaN,boundaries_modelo_prefeitura_mage,1_section_cabecalho_nf,1 - CABECALHO,NaN,0.0,0.0,2067.0,380.0,2067.0,380.0,14.803272
3,4,mage,4,todos,frame,red,100.0,yes,NaN,1_section_cabecalho_nf,1_frame_prefeitura_nf,1 - CABECALHO,NaN,406.0,0.0,1540.0,380.0,1030.0,380.0,NaN
4,5,mage,5,todos,sframe_field,NaN,NaN,no,string,1_frame_prefeitura_nf,nome_prefeitura,1 - CABECALHO,PREFEITURA,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,225,pedro_aldeia,73,todos,field_box,purple,80.0,yes,string,5_frame_inf_criticas,issqn_retido,9. OUTRAS INFORMAÇOES / CRITICAS,ISSQN RETIDO,1066.0,2410.0,1328.0,2521.0,262.0,111.0,NaN
225,226,pedro_aldeia,74,todos,field_box,purple,80.0,yes,string,5_frame_inf_criticas,local_pretacao_servico,9. OUTRAS INFORMAÇOES / CRITICAS,LOCAL. PRESTAÇÃO SERVIÇO,1328.0,2410.0,1638.0,2521.0,310.0,111.0,NaN
226,227,pedro_aldeia,75,todos,field_box,purple,80.0,yes,string,5_frame_inf_criticas,local_incidencia,9. OUTRAS INFORMAÇOES / CRITICAS,LOCAL INCIDÊNCIA,1638.0,2410.0,1922.0,2521.0,284.0,111.0,NaN
227,228,pedro_aldeia,76,todos,frame,red,100.0,yes,NaN,6_section_inf_complementares_criticas,5_frame_observacao,10. OBSERVACOES,uma observação,148.0,2515.0,1922.0,2676.0,1774.0,155.0,NaN


### Funcoes de imagem e extracao

In [26]:
### PRIMEIRAS FUNCOES

# 1. Interacao para pesquisar prefeitura
def pesquisa_texto(texto):
    nome_prefeitura_match = re.search(r'PREFEITURA (.+)', texto)
    if nome_prefeitura_match:
        is_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
        
        return  is_prefeitura
    else:
        raise ValueError("Nao consegui pesquisar")

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

# 3. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# Funcao importante - process_line
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None



# Funcao de conversao e resize do documento
def convertResize_analise(nome_documento, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(nome_documento)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name


# Funcao de pesquisa de modelo
def pequisaModel(image_name):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1)
            return  nome_prefeitura
        else:
            raise ValueError("Nao acho nome de prefeitura")
        
# 1. Funcao de conversao e resize do documento
def convertResize(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    # 4. Verifica se ha mais que uma pagina
    if len(pages) > 1:
        raise ValueError("Erro, documento com mais de uma página")
    else:
        # 5. Iterar pelas páginas e redimensionar
        resized_pages = []
        for page in pages:
            resized_page = page.resize((2067, 2923))
            resized_pages.append(resized_page)
            
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name



# 2. Pesquisa prefeitura no documento
def pequisaModel(image_name):

    # 1. Definindo as coordenadas do frame
    x0 = 406
    y0 = 0
    x1= 1540
    y1 = 380

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')

    # 5. Interacao para pesquisar prefeitura
    for value in values:
        nome_prefeitura_match = re.search(r'PREFEITURA (.+)', value)
        if nome_prefeitura_match:
            nome_prefeitura = "PREFEITURA " + nome_prefeitura_match.group(1) 
            return  nome_prefeitura        
        


# 3. Ajusta o filename tirando caracteres especiais 
def conv_filename(title):
    
    # Divida o título em nome e extensão
    name, extension = title.rsplit('.', 1) if '.' in title else (title, "")

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    # Adicione a extensão de volta, se houver
    if extension:
        filename += '.' + extension.lower()

    return filename

# 4. Ajusta o filename tirando caracteres especiais e a
def conv_filename_no_ext(title):
    # Divida o título em nome e extensão (mas ignore a extensão)
    name = title.rsplit('.', 1)[0] if '.' in title else title

    # Remova acentos e caracteres especiais do nome
    name = normalize('NFKD', name).encode('ASCII', 'ignore').decode('ASCII')
    
    # Substitua espaços e hífens por sublinhados
    filename = name.replace(' ', '_').replace('-', '_')

    # Remova quaisquer outros caracteres não alfanuméricos, exceto sublinhados
    filename = re.sub(r'[^\w_]', '', filename)

    # Converter para minúsculas
    filename = filename.lower()

    return filename  

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 


# 5. Verifica se PDF e pesquisavel ou nao e grava metadados dele
def is_pdf_searchable_analise(pdf_path):
    try:
        pdf_document = fitz.open(pdf_path)
        pages = pdf_document.page_count
        is_searchable = all(page.get_text("text") != "" for page in pdf_document)
        dados_pdf = pdf_document.metadata
        pdf_document.close()
        return is_searchable, dados_pdf, pages
    except Exception as e:
        print(f"Erro ao verificar o PDF: {e}")
        return False

# 2. Efetua OCR no documento (area parao do texto da NF)
def ocr_RasterPDF_free(image_name, vx0, vy0, vx1, vy1):
    
    analise_pesquisa_nf = {}
    # 1. Definindo as coordenadas do frame
    x0 = vx0
    y0 = vy0
    x1= vx1
    y1 = vy1

    # 2. Definir frame_image
    frame_image = image_name.crop((x0, y0, x1, y1))

    # 3. Extraia texto usando OCR com configuração de idioma padrão para este frame
    extracted_text_frame = pytesseract.image_to_string(frame_image, lang='por', config=tessdata_dir_config).strip()

    # 4. Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
    values = extracted_text_frame.split('\n')
    return values, extracted_text_frame 

# Funcao importante - process_line




def convertResizeAnalise_1page(doc2convert, document_path, image_resized_path):
    
    """# 1. remocao do sufixo .pdf
    if doc2convert.split(".")[1].islower():
        nameImage= doc2convert.removesuffix(".pdf")
    else:
        nameImage= doc2convert.removesuffix(".PDF")"""
    
    # 2. construo um novo nome para o documento imagem
    image_resized_name = os.path.join(f'{image_resized_path}/{str(doc2convert)}.jpg')
    
    # 3. Conversao para imagem
    pages = convert_from_path(document_path, 500, poppler_path=poppler_path)
    
    resized_pages = []
    for page in pages:
        resized_page = page.resize((2067, 2923))
        resized_pages.append(resized_page)
        resized_pages[0].save(image_resized_name, 'JPEG')
        
    return resized_pages[0], image_resized_name

# 2. Leitura do arquivo CSV e criação do dicionário modelos
def create_model_dictionary(model_dict_path):
    model_dictionary = {}
    with open(model_dict_path, 'r') as csvfile:
        csvreader = csv.DictReader(csvfile)
        for row in csvreader:
            prefeitura_name = row['prefeitura']
            model_name = row['model']

            if prefeitura_name not in model_dictionary:
                model_dictionary[prefeitura_name] = model_name
            
            #model_dictionary[prefeitura_name].append(model_name)
    
    return model_dictionary


def format_number(number_str):
    # Check for percentage and handle it
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # You can multiply by 100 here if needed

    # Check if the string contains "R$" or a comma, indicating the original format
    if 'R$' in number_str or ',' in number_str:
        # Original format: Remove 'R$', replace dots with nothing, and replace commas with dots
        number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    else:
        # New format: Extract only the numeric part using regex
        number_str = re.findall(r'[\d\.]+', number_str)[-1]

    return float(number_str)

# Funçao de formatacao de numeros
def format_number2(number_str):
    number_str = number_str.replace('R$', '').replace('.', '').replace(',', '.')
    if '%' in number_str:
        number_str = number_str.replace('%', '')
        return float(number_str)  # multiplica por 100 para fields %
    return float(number_str)


def extract_fields_box(modelo, father_value, section):

    data_box_valores = {}
    data_box_valores['secao'] = section
    filtered_boxes_info = field_boxes_info[(field_boxes_info['father'] == father_value) & (field_boxes_info['model'] == modelo)]
    # Iterate nas informações dos boxes de fields e extraia o texto de cada field
    for index_field, row_field in filtered_boxes_info.iterrows():
        x0, y0, x1, y1 = row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        #print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference'], row_field['x0'], row_field['y0'], row_field['x1'], row_field['y1'] ))
        # Divida o texto por nova linha e mantenha apenas a última parte (assume que o valor está sempre no final)
        value = extracted_text_box.split('\n')[-1]
        # Remova qualquer espaço em branco à esquerda ou à direita
        value = value.strip()
        if row_field['t_value'] == 'number':
            # Formate o valor usando a função format_number
            #print("vou verificar valor")
            value = format_number2(value)
            #print(value)
        # Armazene o texto extraído com o rótulo correspondente
        label = row_field['label']
        data_box_valores[label] = value
        
    return data_box_valores

# move NF processadas ok
def move_raster_pdf(document_path, raster_pdf_path, batch_name, doc2convert):
    # Determine the destination directory
    destination_dir = os.path.join(raster_pdf_path, batch_name)

    # Check if the destination directory exists; if not, create it
    if not os.path.exists(destination_dir):
        os.makedirs(destination_dir)

    # Determine the destination path including the filename
    destination_path = os.path.join(destination_dir, os.path.basename(document_path))

    # Move the file from the source path to the destination path
    try:
        shutil.move(document_path, destination_path)
        print(f"Sucesso ao mover: {document_path} para: {destination_path}")
        return True, destination_path, None  # Success, destination path, no error
    except Exception as e:
        error_message = f"Erro ao mover: {document_path} para: {destination_path}: {str(e)}"
        print(error_message)
        return False, None, error_message  # Failure, no destination path, error message
        

### cricao dos dados ambiente

In [27]:
#### IMPORTANTE - NRO BATCH PARA TESTE    0 = PDF_PESQUISAVEL | 1 = RASTER_PDF

i_test = 1

tipo_pdf = []
tipo_pdf.append('PDF_PESQUISAVEL')
tipo_pdf.append('RASTER_PDF')
tipo_pdf[i_test]


# Tratamento do Path de ORIGEM DO DOCUMENTOS PARA TESTE QUE SERAO MOVIDOS
list_path_test = []
list_path_test.append("pipeline_extracao_documentos/4_area_testes/pdf_pesquisavel_4_test")
list_path_test.append("pipeline_extracao_documentos/4_area_testes/raster_pdf_4_test")
list_path_test[i_test]

# Frame para teste
i_frame = 0

frames_pesquisa = []
# Filtrar o DataFrame para incluir apenas linhas onde a coluna "model" oriundo de: modelo
filtered_frames_info = frames_info[frames_info['model'] == model]
for index_frame, row_frame in filtered_frames_info.iterrows():
    frame_name = row_frame['label']
    frames_pesquisa.append(frame_name)

# Nome Batch
batch_name = "Batch_" + str(tipo_pdf[i_test]) + "_" + str(i_frame)

# Nome do arquivo json
nome_formado_json = batch_name +".json"

# Listagem dos frames de pesquisa
i = 0
for frame in frames_pesquisa:
    print(f'seq ={i:>3} | {frame}')
    i += 1
    
if frames_pesquisa[i_frame]:
    print(f'\n\nDados do teste: batch_name: {batch_name} | frame: {frames_pesquisa[i_frame]} | model: {model} | tipo_pdf: {tipo_pdf[i_test]}')
    
    
######### PATHS
#1. path formado para busca de pdfs recursiva
root_doc_analise = os.path.join(documentos_extracao_path, batch_name)

#2. path para documentos teste RASTER PDF (ATRIBIDO DA LISTA)
path_test_pdf = list_path_test[1]

#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)

#Listando paths utilizados
#print(f'\nroot_doc_analise: {root_doc_analise}\npath_test_pdf: {path_test_pdf}\njson_file_path: {json_file_path}')       

seq =  0 | 1_frame_prefeitura_nf
seq =  1 | 1_frame_dados_nf
seq =  2 | 2_frame_cnpj_prestador
seq =  3 | 2_frame_inscricao_prestador
seq =  4 | 2_frame_dados_prestador
seq =  5 | 3_frame_cnpj_tomador
seq =  6 | 3_frame_inscricao_tomador
seq =  7 | 3_frame_dados_tomador
seq =  8 | 4_frame_descricao_totais
seq =  9 | 4_frame_valor_total
seq = 10 | 4_frame_cnae_itens_servico
seq = 11 | 5_frame_valores_impostos
seq = 12 | 5_frame_dados_complementares
seq = 13 | 5_frame_inf_criticas
seq = 14 | 5_frame_observacao


Dados do teste: batch_name: Batch_RASTER_PDF_0 | frame: 1_frame_prefeitura_nf | model: pedro_aldeia | tipo_pdf: RASTER_PDF


### Modelo de iteracao

### chamadas de funcao de modelo

In [28]:
# 3. Cria os valores 
def process_line(value, reference, label):
    name_match = re.search(fr'{reference} (.+)', value)
    if name_match:
        extracted_value = reference + " " + name_match.group(1)
        return {label: extracted_value}
    return None


secao = "1 - CABECALHO"
f_frame_name = "1_frame_prefeitura_nf"


#4. Extrai prefeitura
def extract_prefeitura(model, father, values):
    
    tipo = "sframe_field"
    data_extrated_prefeitura = {}
    #print(tipo)

    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['father'] == father) & (frames_nf_v4_df['type'] == tipo)]

    for index_sframe, row_sframe in filtered_frames_nf_v4_df.iterrows():
        
        label_value = row_sframe['label']
        
        #print("label_value", label_value)
        
        if label_value == "nome_prefeitura":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
        elif label_value == "secretaria":
            reference_value = row_sframe['reference']
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result) 
        elif label_value == "tipo_nota_fiscal":
            reference_value = row_sframe['reference']  
            for value in values:
                result = process_line(value, reference_value, label_value)
                if result:
                    data_extrated_prefeitura.update(result)
                    
    return data_extrated_prefeitura

# 2. Extracao OCR
def extract_text_from_coordinates(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

# 1. funçao basica de modelo 
def executa_model_frame(model, secao, father_name):

    data_dados_frame = {}
    
    tipo = "frame"
    filtered_frames_nf_v4_df = frames_nf_v4_df[(frames_nf_v4_df['model'] == model) & (frames_nf_v4_df['label'] == f_frame_name) & (frames_nf_v4_df['type'] == tipo)]

    for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_frame = extract_text_from_coordinates(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        frame_seq = row_frame['seq']
        frame_model = row_frame['model']
        frame_label = row_frame['label']
        frame_type = row_frame['type']
        frame_section = row_frame['section_json']
        frame_reference = row_frame['reference']
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        #print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
        
    return extracted_text_frame




In [29]:
# Sao iguais 
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text


def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited


#1. funcao: find_value_after_keyword_out_frame_up
def find_value_after_keyword_out_frame_up(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
        if text_list[index + 1] not in default_keyword_list:
            return text_list[index + 1]
        else:
            return None
    except ValueError:
        if default_keyword_list:
            for default_keyword in default_keyword_list:
                if default_keyword in text_list:
                    # Caso especial para 'Nome/Razão Social:'
                    if keyword == 'Nome/Razão Social:':
                        return text_list[0]
        return None
    
#2. find_value_after_keyword_out_frame_down  
def find_value_after_keyword_out_frame_down(keyword, text_list, default_keyword_list=None):
    try:
        index = text_list.index(keyword)
        # Verifica se o índice seguinte está dentro da lista
        if index + 1 < len(text_list):
            # Verifica se o valor seguinte não é outra keyword da lista default_keyword_list
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    except ValueError:
        if default_keyword_list:
            try:
                index = text_list.index(default_keyword_list[-1])
                return text_list[index - 1]
            except ValueError:
                return None
        else:
            return None
        
#3. find_value_after_keyword_fuzz
def find_value_after_keyword_fuzz(keyword, text_list, default_keyword_list=None, fuzziness_threshold=80):
    closest_match = None
    closest_match_score = 0
    
    for i, text in enumerate(text_list):
        score = fuzz.ratio(keyword, text)
        
        if score > closest_match_score:
            closest_match_score = score
            closest_match = text
        
        if closest_match_score > fuzziness_threshold:
            break

    if closest_match_score > fuzziness_threshold:
        index = text_list.index(closest_match)
        if index + 1 < len(text_list):
            if text_list[index + 1] not in default_keyword_list:
                return text_list[index + 1]
            else:
                return None
        else:
            return None
    else:
        return None  
    



def pesquisa_keyword(string_pesquisa, text_splited, keyword_list):

    resultado_extraido_fuzz = find_value_after_keyword_fuzz(string_pesquisa, text_splited, keyword_list)

    if resultado_extraido_fuzz == None:
        resultado_extraido_frame_up = find_value_after_keyword_out_frame_up(string_pesquisa, text_splited, keyword_list)
        if resultado_extraido_frame_up == None:
            resultado_extraido_frame_down = find_value_after_keyword_out_frame_down(string_pesquisa, text_splited, keyword_list)
            resultado_extraido = resultado_extraido_frame_down
        else:
            resultado_extraido = resultado_extraido_frame_up
    else:
        resultado_extraido = resultado_extraido_fuzz           
    #print(resultado_extraido)
    return resultado_extraido


def cabecalho_prefeitura():
    valor_dict = {}
    dados_prefeitura = {}
    f_frame_name = "1_frame_prefeitura_nf"
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)
    text_splited = texto.split('\n')
    
    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
    if valor_dict:
        dados_prefeitura.update(valor_dict)
        
        
    return dados_prefeitura 
                
def cabecalho_dados():

    valor = {}   
    f_frame_name = "1_frame_dados_nf"
    
    dadinho_dados_nf = {}
    
    # 1. funçao basica de modelo 
    texto = executa_model_frame(model, secao, f_frame_name)    
    text_splited = texto_extraido(texto)
    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

    string_pesquisa = "Número da Nota:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
    dadinho_dados_nf['numero_nota_fiscal'] = texto


    string_pesquisa = "Competência:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['competencia'] = texto
    
    
    string_pesquisa = "ata e Hora da Emissão:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['dt_hr_emissao'] = texto
    
    
    string_pesquisa = "Código Verificação:"
    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
    dadinho_dados_nf['codigo_verificacao'] = texto
    
    return dadinho_dados_nf      

In [30]:
def extract_fields_prestador_cnpj(text): # Função para extrair campos e valores dentro de um retângulo
    
    
    nf_data_prestador_cnpj = {}
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_prestador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_prestador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))


    # Extrair Telefone
    telefone_str = None
    
    #telefone_match = re.search(r'Telefone:\s+([0-9.\s-])', text)
    telefone_match = re.search(r'Telefone:\s+([0-9.\s-]+)', text)
    if telefone_match: 
        telefone_str = telefone_match.group(1)
        # Remover quebras de linha
        telefone_str = telefone_str.replace('.', '')
        telefone_str = telefone_str.replace('\n', '')
                
        nf_data_prestador_cnpj['telefone'] = telefone_str
    else:
        nf_data_prestador_cnpj['telefone'] = None   
    
    
    return nf_data_prestador_cnpj 


def extract_fields_tomador_cnpj(text):
    nf_data_tomador_cnpj = {}
    
    # Extrair CPF/CNPJ com máscara 1
    if "CPF/CNPJ:" in text:
        cpf_cnpj_formatado_match = re.search(r'(\d{2}\.\d{3}\.\d{3}/\d{4}-\d{2})', text)
        if cpf_cnpj_formatado_match:
                        nf_data_tomador_cnpj['cpf_cnpj_com_mascara'] = cpf_cnpj_formatado_match.group(1)
                        nf_data_tomador_cnpj['cpf_cnpj_sem_mascara'] = re.sub(r'\D', '', cpf_cnpj_formatado_match.group(1))

    
    # Extrair Telefone
    telefone_match = re.search(r'Telefone:\s+(.+)', text)
    if telefone_match:
        telefone_str = telefone_match.group(1)
        if telefone_str == 'Inscrição Estadual:':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
        elif telefone_str == '':
            nf_data_tomador_cnpj['telefone'] = None  # Valor padrão quando não há correspondência
                    
        else:    
            nf_data_tomador_cnpj['telefone'] = telefone_match.group(1)
            
    
    # Extrair Inscrição Municipal
    inscricao_municipal_match = re.search(r'Inscrição Municipal:\s+(.+)', text)
    if inscricao_municipal_match:
        inscricao_municipal_str = inscricao_municipal_match.group(1)
        if inscricao_municipal_str == "Telefone:": 
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = inscricao_municipal_str
    
    insc_municipal_match = re.search(r'INSC:MUNICIPAL:\s+(.+)', text)
    if insc_municipal_match:
        insc_municipal_str = insc_municipal_match.group(1)
        if insc_municipal_str == "Telefone:":
            nf_data_tomador_cnpj['inscricao_municipal'] = None
        else:    
            nf_data_tomador_cnpj['inscricao_municipal'] = insc_municipal_str
    else:
        nf_data_tomador_cnpj['inscricao_municipal'] = None
            
    
    return nf_data_tomador_cnpj

In [12]:
def texto_extraido(texto):
    #0. Tratamento da string
    text_splited = texto.split('\n')
    text_splited = [s.replace(":", "") for s in text_splited]
    text_splited = [x for x in text_splited if x.strip()]
    text_splited = [s.replace(";", "").strip() for s in text_splited] #depende da situaçao
    return text_splited

In [31]:
mage_cnae_x_item_servico_df = pd.read_excel('pipeline_extracao_documentos/6_geral_administacao/datasets/MAGE_CNAE_X_ITEM_SERVICO_V1.xlsx')

# Creating a dictionary for CNAE codes and descriptions
cnae_dict = dict(zip(mage_cnae_x_item_servico_df['cnae'], mage_cnae_x_item_servico_df['descricao_cnae']))
item_servico_dict = dict(zip(mage_cnae_x_item_servico_df['item_servico'], mage_cnae_x_item_servico_df['descricao_item_servico']))



# Função para extrair número da string
def extract_number(text):
    match = re.search(r'\b\d+(\.\d+)?\b', text)
    if match:
        return match.group(0)
    else:
        return None

In [32]:
def extract_dados_comple_obs(modelo, frame_father, section):
    
    data_dados_complementares = {}
    #frame_label = frame_father
    
    # 1. Filtrando o frames_info para buscar os dados de corte
    filtered_frames_info = frames_info[(frames_info['label'] == frame_father) & (frames_info['model'] == modelo)]

    # 2. Filtrando o sframe_fields_info para buscar os dados dos campos que estao nos frames
    filtered_sframe_fields_info = sframe_fields_info[(sframe_fields_info['father'] == frame_father) & (sframe_fields_info['model'] == modelo)]

    for index_frame, row_frame in filtered_frames_info.iterrows():
        
        x0, y0, x1, y1 = row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1']
        extracted_text_box = extract_text_from_frame(image_2work, (x0, y0, x1, y1), tessdata_dir_config)
        
        print("{:<5} {:<10} {:<30} {:<20} {:<20} {:<7} {:<7} {:<7} {:<7}".format(row_frame['seq'], row_frame['model'], row_frame['father'], row_frame['label'], row_frame['reference'], row_frame['x0'], row_frame['y0'], row_frame['x1'], row_frame['y1'] ))
        for index_field, row_field in filtered_sframe_fields_info.iterrows():
            #print("{:<5} {:<10} {:<30} {:<20} {:<20}".format(row_field['seq'], row_field['model'], row_field['father'], row_field['label'], row_field['reference']))
            
            if frame_father == "5_frame_dados_complementares":
                nf_data_dados_complementares = {}
                nf_data_dados_complementares['section'] = section
                
                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^DADOS COMPLEMENTARES', '', extracted_text_box, count=1)
                if text == '':
                    text = None
                    nf_data_dados_complementares['dados_complementares'] = text
                else:    
                    # Extrair texto dentro do retângulo
                    nf_data_dados_complementares['dados_complementares'] = text.strip()
                    
                return nf_data_dados_complementares                
                
            elif frame_father == "5_frame_observacao":
                nf_data_observacao = {}
                nf_data_observacao['section'] = section 
                                # Remove a primeira ocorrência de "Observação:"
                text = re.sub(r'^Observação:', '', extracted_text_box, count=1)

                # Remover quebras de linha
                text = text.replace('\n', ' ')

                # Extrair texto dentro do retângulo
                nf_data_observacao['observacao'] = text.strip()
                
                return nf_data_observacao   

In [33]:
batch_name = "Batch_15"

In [34]:
root_doc_analise = "pipeline_extracao_documentos/2_documentos_para_extracao/Batch_15"

## Processamento do Pipeline 

In [35]:
erros = {}

# 1. Leitura recursiva de diretorios e arquivos a partir de root
pdf_info = {}  # Dicionário para armazenar informações sobre PDFs

nf_data_servico = {}#VERIFICAR
analise_doc_nf = {} #VERIFICAR
file_data = [] #VERIFICAR

list_document_pages = []
#nro_nota = 0
# TEMP
# Nome do arquivo json
nome_formado_json = batch_name +".json"
#3. path formado para nome do arquivo json
json_file_path = os.path.join(json_path, nome_formado_json)
#root_doc_analise = os.path.join(documentos_extracao_path, batch_name)
#print(root_doc_analise)
i = 1
for root, dirs, files in os.walk(root_doc_analise):
    dir_name = os.path.basename(root)
    #print(dir_name)
    for file in files:
        
        if file.lower().endswith('.pdf'):
            doc2convert = file
            document_path_1 = os.path.join(root, file)
            pdf_document = fitz.open(document_path_1)
            #page_number = 0  # Defina o número da página que deseja analisar
            #page = pdf_document[page_number]
            
            documento_pdf = True
            pesquisavel, metadados, paginas = is_pdf_searchable_analise(document_path_1)
            #print(f'\nTeste nro: {i} | doc: {file} | pdf?: {documento_pdf} | pesquisavel?: {pesquisavel} | paginas: {paginas}\n')
                
            if not pesquisavel:
                image_2work, name_image_2work = convertResizeAnalise_1page(file, document_path_1, image_resized_path)
                
                secao = "1 - CABECALHO"
                try:
                    nro_nota = 0
                    data_cabecalho = {}
                    data_cabecalho['secao'] = secao
                    valor_dict = {}
                    dados_prefeitura = {}
                    f_frame_name = "1_frame_prefeitura_nf"
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto.split('\n')
                    
                    valor_dict = extract_prefeitura(model, f_frame_name, text_splited)
                    if valor_dict:
                        dados_prefeitura.update(valor_dict)
                    valor = {}   
                    f_frame_name = "1_frame_dados_nf"
                    dadinho_dados_nf = {}
                    # 1. funçao basica de modelo 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Número da Nota:', 'Competência:', 'Data e Hora da Emissão:', 'Código Verificação:']

                    string_pesquisa = "Número da Nota:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)         
                    dadinho_dados_nf['numero_nota_fiscal'] = texto
                    nro_nota = texto
                    
                    string_pesquisa = "Competência:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['competencia'] = texto
                    
                    string_pesquisa = "ata e Hora da Emissão:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['dt_hr_emissao'] = texto
                    
                    string_pesquisa = "Código Verificação:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    dadinho_dados_nf['codigo_verificacao'] = texto
                    
                    data_cabecalho.update(dados_prefeitura)
                    data_cabecalho.update(dadinho_dados_nf)
                except Exception as e:
                    erros_cabecalho = {}
                    err_msg = f"Erro de processo cabecalho: {e}"
                    erros['documento'] = file
                    erros_cabecalho['secao'] = secao
                    erros_cabecalho['erro'] = err_msg
                    erros.update(erros_cabecalho)                
               
                
                
                
                secao = "2. PRESTADOR DE SERVIÇO"
                try:
                    f_frame_name = "2_frame_cnpj_prestador"
                    data_prestador = {}
                    prestador_inscricao = {}
                    data_prestador['secao'] = secao
                    prestador_cnpj_value = {}
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    prestador_cnpj_value = extract_fields_prestador_cnpj(texto)
                    if prestador_cnpj_value:
                        data_prestador.update(prestador_cnpj_value)
                except Exception as e:
                    erros_cnpj_prestador = {}
                    err_msg = (f"Erro prestador cnpj: {e}")
                    erros_cnpj_prestador['secao'] = secao
                    erros_cnpj_prestador['erro'] = err_msg
                    erros.update(erros_cnpj_prestador)       
                    
                try:
                    f_frame_name = "2_frame_inscricao_prestador" 
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    keyword_list = ['Inscrição Municipal:', 'Inscrição Estadual:']
                    string_pesquisa = "Inscrição Municipal:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['prestador_inscricao'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_inscricao['inscricao_estadual'] = texto
                    data_prestador.update(prestador_inscricao)
                except Exception as e:
                    erros_inscricao_prestador = {}
                    err_msg = (f"Erro de processo inscricao prestador: {e}")
                    erros_inscricao_prestador['secao'] = secao
                    erros_inscricao_prestador['erro'] = err_msg
                    erros.update(erros_inscricao_prestador)

                try:
                    f_frame_name = "2_frame_dados_prestador"
                    prestador_dados_value = {}
                    
                    keyword_list = ['Nome/Razão Social:', 'Nome de Fantasia:', 'Endereço:', 'E-mail:']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['razao_social'] = texto

                    string_pesquisa = "Nome de Fantasia:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['nome_fantasia'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['endereco'] = texto
                    
                    string_pesquisa = "E-mail:"
                    #texto = executa_model_frame(model, secao, f_frame_name)
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    prestador_dados_value['email'] = texto
                    data_prestador.update(prestador_dados_value)
                    
                except Exception as e:
                    print(f"Erro prestador dados: {e}") 
                    
                
                secao = "3. TOMADOR DE SERVIÇO"
                try:
                    data_tomador = {}
                    tomador_cnpj_value = {}
                    data_tomador['secao'] = secao
                    f_frame_name = "3_frame_cnpj_tomador"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto) 
                    tomador_cnpj_value = extract_fields_tomador_cnpj(texto)
                    if tomador_cnpj_value:
                        data_tomador.update(tomador_cnpj_value)                  
                except Exception as e:
                    print(f"Erro tomador cnpj: {e}")
                    
                f_frame_name = "3_frame_inscricao_tomador"    
                try:
                    data_tomador_inscricao = {}
                    keyword_list = ['RG:', 'Inscrição Estadual:']
                    string_pesquisa = "RG:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['rg'] = texto
                    
                    string_pesquisa = "Inscrição Estadual:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_inscricao['inscricao_estadual'] = texto
                    data_tomador.update(data_tomador_inscricao)
                except Exception as e:
                    print(f"Erro tomador inscricao: {e}")
                    
                f_frame_name = "3_frame_dados_tomador"
                try: 
                    data_tomador_dados = {}   
                    keyword_list = ['Nome/Razão Social:', 'Endereço:', 'E-mail']
                    string_pesquisa = "Nome/Razão Social:"
                    texto = executa_model_frame(model, secao, f_frame_name)    
                    text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['razao_social'] = texto
                    
                    string_pesquisa = "Endereço:"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['endereco'] = texto
                    
                    string_pesquisa = "E-mail"
                    #texto = executa_model_frame(model, secao, f_frame_name)    
                    #text_splited = texto_extraido(texto)
                    texto = pesquisa_keyword(string_pesquisa, text_splited, keyword_list)
                    data_tomador_dados['email'] = texto
                    
                    data_tomador.update(data_tomador_dados)
                except Exception as e:
                    print(f"Erro tomador dados: {e}") 
                    
                secao = "4. DESCRIMINACAO DOS SERVIÇOS"
                try:
                    data_servico = {}
                    data_servico['secao'] = secao
                    f_frame_name = "4_frame_descricao_totais"
                    texto = executa_model_frame(model, secao, f_frame_name)
                    text = texto.replace('\n', ' ')
                    label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                    if text.startswith(label):
                        text = text[len(label):].strip()
                    data_servico['discriminacao_servicos'] = text 

                except Exception as e:
                    print(f"Erro descricao servico: {e}")
                     
                secao = "5. VALOR TOTAL"
                try:
                    data_valor_total = {}
                    data_valor_total['secao'] = secao
                    f_frame_name = "4_frame_valor_total"   
                    text = executa_model_frame(model, secao, f_frame_name)  
                    valor_total_match = re.search(r'R\$ ([\d,.]+)', text)
                    if valor_total_match:
                        valor_total_sem_formatacao = valor_total_match.group(1).replace('.', '').replace(',', '.')
                        data_valor_total['valor_total_nota'] = float(valor_total_sem_formatacao)
                except Exception as e:
                    print(f"Erro valor total: {e}")

                secao = "6. CNAE e Item da Lista de Serviços"
                try:
                    data_CNAE = {}
                    
                    data_CNAE['secao'] = secao
                    f_frame_name = "4_frame_cnae_itens_servico"   
                    Texto_extraido = executa_model_frame(model, secao, f_frame_name)
                    text_splited = Texto_extraido.split('\n')
                    # Processando CNAE
                    cnae_line = [line for line in text_splited if 'CNAE' in line][0]
                    cnae_number = int(extract_number(cnae_line))
                    cnae_value = cnae_dict.get(cnae_number, "Valor não encontrado")
                    cnae_value = cnae_value.upper()
                    cnae_value = str(cnae_number) + " - " + cnae_value
                    data_CNAE['cnae'] = cnae_value
                    
                    item_servico_line = [line for line in text_splited if 'Item da Lista de Serviços' in line][0]
                    item_servico_number = float(extract_number(item_servico_line))
                    item_servico_value = item_servico_dict.get(item_servico_number, "Valor não encontrado")
                    item_servico_value = item_servico_value.upper()
                    item_servico_value = str(item_servico_number) + " - " + item_servico_value
                    
                    data_CNAE['item_lista_servicos'] = item_servico_value
                except Exception as e:
                    print(f"Erro busca cnae: {e}")                    

                
                secao = "8. DADOS COMPLEMENTARES"
                try:
                    data_valores = {}
                    
                    data_valores['secao'] = secao
                    f_frame_name = "5_frame_valores_impostos"   
                    
                    result = extract_fields_box(model, f_frame_name, secao)
                    if result:
                        data_valores.update(result)
            
                    # secao: 8 - DADOS COMPLEMENTARES"
                    data_dados_complementares = {}
                    f_frame_name  = "5_frame_dados_complementares"
                    section = "8. DADOS COMPLEMENTARES"

                    data_dados_complementares = extract_dados_comple_obs(model, f_frame_name, section)                                           
                                            
                                            
                    # secao: 9 - OUTRAS INFORMAÇOES / CRITICAS
                    data_outras_informacoes = {}
                    father_value = "5_frame_inf_criticas"
                    section = "9. OUTRAS INFORMAÇOES / CRITICAS"

                    result = extract_fields_box(model, father_value, section)
                    if result:
                        data_outras_informacoes.update(result)                        
                                        
                    # secao: 10. OBSERVACOES
                    data_observacao = {}
                    f_father = "5_frame_observacao"
                    section = "10. OBSERVACOES"

                    data_observacao = extract_dados_comple_obs(model, f_father, section)
                except Exception as e:
                    print(f"Erro valores complementares: {e}")   
                
                nome_arquivo = file
                #pdf_info["diretorio"] = os.path.basename(root)
                try:
                    pdf_info[nro_nota] = {
                    "dados_NF_PDF": {
                        "data_cabecalho": data_cabecalho,
                        "data_prestador": data_prestador,
                        "data_tomador": data_tomador,
                        "data_servico": data_servico,
                        "data_valor_total": data_valor_total,
                        "data_CNAE": data_CNAE,
                        "data_valores": data_valores,
                        "data_dados_complementares": data_dados_complementares,
                        "data_outras_informacoes": data_outras_informacoes,
                        "data_observacao": data_observacao,
                    },
                    "diretorio": dir_name, #os.path.basename(root)
                    "nome_arquivo": nome_arquivo,
                    "Batch": batch_name,
                }
                except Exception as e:
                    print(f"Erro ao gerar o json: {e}")
                pdf_document.close()
                #print(pdf_info)
                #if paginas == 1:
                if paginas > 1000:
                    if i == 1000: #Define quantidade de tratamento de documentos raster PDF
                        break
            i +=1 
                
# Salvando as informações em um arquivo JSON (novo formato nome arquivo V2)
with open(json_file_path, "w", encoding='utf-8') as json_file:
    json.dump(pdf_info, json_file, ensure_ascii=False, indent=4)

print(f"As informações foram salvas em {json_file_path}")

#print(pdf_info)  

67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_observacao   uma observação       148.0   2515.0  1922.0  2676.0 
67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_observacao   uma observação       148.0   2515.0  1922.0  2676.0 
67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_observacao   uma observação       148.0   2515.0  1922.0  2676.0 
67    pedro_aldeia 6_section_inf_complementares_criticas 5_frame_dados_complementares nan                  148.0   2273.0  1925.0  2377.0 
76    pedro_aldeia 6_section_inf_complementares_cri

## Desenha Box

In [ ]:
name_image_2work = "pipeline_extracao_documentos/6_geral_administacao/images/processadas/Doria Marinho 0297 Raquel.pdf.jpg"

In [40]:
# Define a mapping for color names to RGB values
color_mapping = {
    "red": (255, 0, 0),
    "purple": (128, 0, 128),
    "orange": (255, 165, 0),
    "green": (0, 128, 50),
    "blue": (0, 0, 255),
    "yellow": (255, 255, 0)
}

# Reload the image to start fresh
image = Image.open(name_image_2work)
draw = ImageDraw.Draw(image)

# Define a font size for the labels using the default PIL font
font_size = 100
#font = ImageFont.load_default()

font = ImageFont.truetype("/usr/share/fonts/truetype/ubuntu/Ubuntu-M.ttf", 30, encoding="unic")

# Update the draw_box function to use the larger font size with the default font
def draw_box(row):
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    color = color_mapping.get(row['color'], (0, 0, 0)) # Default to black if color not found
    draw.rectangle([x0, y0, x1, y1], outline=color, width=3)
    label = str(row['label']) if pd.notnull(row['label']) else None # Check for missing label
    if label:
        draw.text((x0 + 5, y0 + 5), label, fill=color, font=font)

# Draw the boundaries
#draw_box(boundaries_info)


def draw_box_model(modelo,
                   boundaries_info=None,
                   sections_info=None,
                   frames_info=None,
                   field_boxes_info=None,
                   draw_boundaries=True,
                   draw_sections=True,
                   draw_frames=True,
                   draw_field_boxes=True):
    
    # Draw boundaries if requested
    if draw_boundaries and boundaries_info is not None:
        filtered_boundaries_info = boundaries_info[boundaries_info['model'] == modelo]
        for index, row in filtered_boundaries_info.iterrows():
            draw_box(row)

    # Draw sections if requested
    if draw_sections and sections_info is not None:
        filtered_sections_info = sections_info[sections_info['model'] == modelo]
        for index, row in filtered_sections_info.iterrows():
            draw_box(row)
            
    # Draw frames if requested
    if draw_frames and frames_info is not None:
        filtered_frames_info = frames_info[frames_info['model'] == modelo]
        for index, row in filtered_frames_info.iterrows():
            draw_box(row)
            
    # Draw field boxes if requested
    if draw_field_boxes and field_boxes_info is not None:
        filtered_field_boxes_info = field_boxes_info[field_boxes_info['model'] == modelo]
        for index, row in filtered_field_boxes_info.iterrows():
            draw_box(row)
    
    # Show the image with selected drawings
    image.show()

In [42]:
# To draw everything
draw_box_model(model, boundaries_info, sections_info, frames_info, field_boxes_info)


[25644:25672:0830/033433.312059:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[25644:25687:0830/033437.517783:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[25644:25687:0830/033437.517824:ERROR:bus.cc(406)] Failed to connect to the bus: Failed to connect to socket /run/dbus/system_bus_socket: No such file or directory
[25644:25672:0830/033437.558832:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[25644:25672:0830/033437.558904:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address type (examples of valid types are "tcp" and on UNIX "unix")
[25644:25672:0830/033437.558917:ERROR:bus.cc(406)] Failed to connect to the bus: Could not parse server address: Unknown address t

In [ ]:
# To draw only boundaries and sections:
draw_box_model(modelo, boundaries_info, sections_info, draw_frames=False, draw_field_boxes=False)


In [ ]:
# To draw only field boxes:
draw_box_model(modelo, field_boxes_info=field_boxes_info, draw_boundaries=False, draw_sections=False, draw_frames=False)


# Outros

In [ ]:
                    # secao: 1 - CABECALHO
                    #seq =  0 | 1_frame_prefeitura_nf
                    if frame_pesquisa == "1_frame_prefeitura_nf":
                        data_cabecalho = {}
                        f_father = "1_frame_prefeitura_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                        
                    #seq =  1 | 1_frame_dados_nf
                    elif frame_pesquisa == "1_frame_dados_nf":
                        data_prestador = {}
                        f_father = "1_frame_dados_nf"
                        section = "1 - CABECALHO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 2. PRESTADOR DE SERVIÇO
                    #seq =  2 | 2_frame_cnpj_prestador
                    elif frame_pesquisa == "2_frame_cnpj_prestador":
                        data_prestador = {}
                        f_father = "2_frame_cnpj_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                        
                    #seq =  3 | 2_frame_inscricao_prestador    
                    elif frame_pesquisa == "2_frame_inscricao_prestador":
                        data_prestador = {}
                        f_father = "2_frame_inscricao_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)       
                    
                    #seq =  4 | 2_frame_dados_prestador    
                    elif frame_pesquisa == "2_frame_dados_prestador":
                        data_prestador = {}
                        f_father = "2_frame_dados_prestador"
                        section = "2. PRESTADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido)
                           
                           
                    # secao: 3 - TOMADOR       
                    #seq =  5 | 3_frame_cnpj_tomador       
                    elif frame_pesquisa == "3_frame_cnpj_tomador": #TBD
                    
                    #seq =  6 | 3_frame_inscricao_tomador 
                    elif frame_pesquisa == "3_frame_inscricao_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_inscricao_tomador"
                        section = "3. TOMADOR DE SERVIÇO"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        print(Texto_extraido) 
                    
                    #seq =  7 | 3_frame_dados_tomador    
                    elif frame_pesquisa == "3_frame_dados_tomador":
                        
                        data_tomador = {}
                        f_father = "3_frame_dados_tomador"
                        section = "3. TOMADOR DE SERVIÇO"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido)
                        
                        
                    # secao: 4 - DESCRIMINACAO DOS SERVIÇOS
                    #seq =  8 | 4_frame_descricao_totais
                    elif frame_pesquisa == "4_frame_descricao_totais":
                        data_servico = {}
                        result = {}
                        f_father = "4_frame_descricao_totais"
                        section = "4. DESCRIMINACAO DOS SERVIÇOS" 
                        vx_0 = 125
                        vy_0 = 1123
                        vx_1 = 1934
                        vy_1 = 1720
                        data_servico['secao'] = section
                        result, texto_extraido = ocr_RasterPDF_free(image_2work, vx_0, vy_0, vx_1, vy_1)
                        nf_data_servico = {}
                        label = "DISCRIMINAÇÃO DOS SERVIÇOS"
                        if texto_extraido.startswith(label):
                            text = texto_extraido[len(label):].strip()
                        data_servico['discriminacao_servicos'] = text 
                        print(data_servico)
                        
                        
                    # secao: 5. VALOR TOTAL
                    #seq =  9 | 4_frame_valor_total   
                    elif frame_pesquisa == "4_frame_valor_total":
                        data_CNAE = {}
                        f_father = "4_frame_valor_total"
                        section = "5. VALOR TOTAL"    
                        Texto_extraido = extract_fields_box_geral(model, f_father, section)
                        print(Texto_extraido) 
                        
                        
                    # secao: 6. CNAE e Item da Lista de Serviços
                    #seq = 10 | 4_frame_cnae_itens_servico     
                    elif frame_pesquisa == "4_frame_cnae_itens_servico":
                        data_CNAE = {}
                        f_father = "4_frame_cnae_itens_servico"
                        section = "6. CNAE e Item da Lista de Serviços"    
                        Texto_extraido = extract_fields_box(model, f_father, section)
                        
                    
                    # secao: 7. VALORES E IMPOSTOS
                    #seq = 11 | 5_frame_valores_impostos  
                    elif frame_pesquisa == "5_frame_valores_impostos":
                        data_valores = {}
                        result = {}
                        father_value = "5_frame_valores_impostos"
                        section = "7. VALORES E IMPOSTOS"

                        result = extract_fields_box(modelo, father_value, section)
                        if result:
                            data_valores.update(result)
                         

                    #seq = 12 | 5_frame_dados_complementares
                    elif frame_pesquisa == "5_frame_dados_complementares": #TDB
                        
                    
                    #seq = 13 | 5_frame_inf_criticas
                    elif frame_pesquisa == "5_frame_inf_criticas": #TDB    
                    
                    
                    # secao: 10. OBSERVACOES  
                    #seq = 14 | 5_frame_observacao  
                    elif frame_pesquisa == "5_frame_observacao":
                        data_observacao = {}
                        f_father = "5_frame_observacao"
                        section = "10. OBSERVACOES"
                        Texto_extraido = extract_dados_from_frame(model, f_father, section)
                        Texto_extraido

In [ ]:
# Sao iguais APAGAR ESSA
def extract_text_from_frame(image, coordinates, config):
    x0, y0, x1, y1 = coordinates
    frame_image = image.crop((x0, y0, x1, y1))
    extracted_text = pytesseract.image_to_string(frame_image, lang='por', config=config).strip()
    return extracted_text

In [ ]:
def executa_model()
i_frame = 0
frames_pesquisa = []
filtered_frames_nf_v4_df = frames_nf_v4_df[frames_nf_v4_df['model'] == model]
for index_frame, row_frame in filtered_frames_nf_v4_df.iterrows():
    frame_seq = row_frame['seq']
    frame_model = row_frame['model']
    frame_label = row_frame['label']
    frame_type = row_frame['type']
    frame_section = row_frame['section_json']
    frame_reference = row_frame['reference']
    frame_father = row_frame['father']
    #if frame_section == "1 - CABECALHO" and frame_type == "frame":
    if frame_section == "1 - CABECALHO":
        frame_father = row_frame['father']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "1_frame_prefeitura_nf" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "1_frame_dados_nf" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "2. PRESTADOR DE SERVIÇO":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "2_frame_cnpj_prestador" and frame_type == "sframe_field":
        #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
        elif frame_father == "2_frame_inscricao_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
        elif frame_father == "2_frame_dados_prestador" and frame_type == "sframe_field":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
            
    elif frame_section == "3. TOMADOR DE SERVIÇO":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "3_frame_cnpj_tomador" and frame_type == "sframe_field":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
            
            elif frame_father == "3_frame_inscricao_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {} 
            elif frame_father == "3_frame_dados_tomador" and frame_type == "sframe_field":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}
                
    elif frame_section == "4. DESCRIMINACAO DOS SERVIÇOS":
            frame_father = row_frame['father']
            frame_id = row_frame['id']
            data_cabecalho = {}
            data_cabecalho['secao'] = frame_section
            if frame_father == "4_frame_descricao_totais" and frame_type == "field_box":
                #if frame_label == "1_frame_prefeitura_nf":
                frame_father = row_frame['father']
                print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
                result = {}  
                
    elif frame_section == "5. VALOR TOTAL":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_valor_total" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}   
            
    elif frame_section == "6. CNAE e Item da Lista de Serviços":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "4_frame_cnae_itens_servico" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
    
    elif frame_section == "8. DADOS COMPLEMENTARES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_valores_impostos" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}
        if frame_father == "5_frame_dados_complementares" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "9. OUTRAS INFORMAÇOES / CRITICAS":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_inf_criticas" and frame_type == "field_box":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {} 
    elif frame_section == "10. OBSERVACOES":
        frame_father = row_frame['father']
        frame_id = row_frame['id']
        data_cabecalho = {}
        data_cabecalho['secao'] = frame_section
        if frame_father == "5_frame_observacao" and frame_type == "sframe_field":
            #if frame_label == "1_frame_prefeitura_nf":
            frame_father = row_frame['father']
            print(f'\fid: {frame_id:>3} | seq: {frame_seq:>3} | model: {frame_model:>8} | type: {frame_type:>15} | Father: {frame_father} label: {frame_label:>30} | section: {frame_section:>20} {frame_reference:>30}')
            result = {}                             
        
        